In [1]:
from metasynth.distribution.regex2 import RegexNewDistribution
from metasynth.distribution.string import RegexDistribution
from metasynth.distribution.util import RegexOptimizer
import pandas as pd
from metasynth import MetaDataset
import wget
from pathlib import Path
import json
import string
import re
import numpy as np

In [2]:
dtypes = {
    "Survived": "category",
    "Pclass": "category",
    "Name": "string",
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category"
}
titanic_fp = Path("titanic.csv")
if not titanic_fp.is_file():
    wget.download("https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv")
df = pd.read_csv(titanic_fp, dtype=dtypes)

In [3]:
regex = RegexDistribution.fit(df["Ticket"])

In [4]:
match_regex = r"\d+"

In [5]:
rx = re.compile(match_regex)

In [6]:
spans = []
cur_solution = []
cur_len = np.zeros(100, dtype=int)
values = df["Ticket"].dropna().values[:10]
for val in values:
    spans.append([match.span() for match in rx.finditer(val)])
    if len(spans) > 0:
        i_cur = np.argmax([x[1]-x[0] for x in spans[-1]])
        cur_span = spans[-1][i_cur]
        cur_solution.append(i_cur)
        cur_len[-(cur_span[0])] += 1
        cur_len[len(val)-cur_span[1]] += 1
    else:
        cur_solution.append(-1)
        cur_len[0] += 1
        cur_len[len(val)] += 1
    
def energy(cur_len):
    left_energy = np.arange(0, 50)*np.log(cur_len[:50]+1)
    right_energy = np.arange(50, 0, -1)*np.log(cur_len[-50:]+1)
    return np.sum(left_energy + right_energy)
print(energy(cur_len))

def energy_rm(length):
    n_i = cur_len[length]
    print(length, n_i)
    return np.abs(length)*np.log(n_i/(n_i+1))

def energy_add(length):
    n_i = cur_len[length]
    print(length, n_i)
    return np.abs(length)*np.log((n_i+2)/(n_i+1))


n_change = 0
for i, val in enumerate(values):
    print(val)
    i_start, i_temp = spans[i][cur_solution[i]]
    i_end = len(val)-i_temp
    print(spans[i], i_end)
    for j, _ in enumerate(spans[i]):
        if j == cur_solution[i]:
            continue
        j_start, j_temp = spans[i][j]
        j_end = len(val)-j_temp
        print(i_start, i_end, j_start, j_end)
        delta_E = energy_rm(-i_start) + energy_rm(i_end) + energy_add(-j_start) + energy_add(j_end)
        print(delta_E)
    print("----------------\n\n")


11.090354888959125
A/5 21171
[(2, 3), (4, 9)] 0
4 0 2 6
-4 1
0 17
-2 0
6 0
2.7725887222397807
----------------


PC 17599
[(3, 8)] 0
----------------


STON/O2. 3101282
[(6, 7), (9, 16)] 0
9 0 6 9
-9 1
0 17
-6 0
9 0
4.1588830833596715
----------------


113803
[(0, 6)] 0
----------------


373450
[(0, 6)] 0
----------------


330877
[(0, 6)] 0
----------------


17463
[(0, 5)] 0
----------------


349909
[(0, 6)] 0
----------------


347742
[(0, 6)] 0
----------------


237736
[(0, 6)] 0
----------------




In [8]:
optimizer = RegexOptimizer(values, spans)
optimizer.cur_solution

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [10]:
optimizer.set_span(0, 0)

In [11]:
optimizer.energy

13.862943611198906

In [12]:
optimizer.optimize()

In [13]:
optimizer.energy

11.090354888959125

In [15]:
optimizer.cur_solution

array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0])